In [1]:
import yfinance as yf

# Create a Ticker object for Apple Inc.
ticker = yf.Ticker("AAPL")

# Fetch historical market data for the last year
historical_data = ticker.history(period="1y")
print(historical_data)

# Fetch basic financials
financials = ticker.financials
#print(financials)

                                 Open        High         Low       Close  \
Date                                                                        
2024-05-24 00:00:00-04:00  187.941069  189.692870  187.164685  189.095657   
2024-05-28 00:00:00-04:00  190.618520  192.101589  188.219750  189.105606   
2024-05-29 00:00:00-04:00  188.727393  191.355104  188.627853  189.404221   
2024-05-30 00:00:00-04:00  189.872035  191.285423  189.742650  190.399567   
2024-05-31 00:00:00-04:00  190.548875  191.673620  189.025999  191.355103   
...                               ...         ...         ...         ...   
2025-05-19 00:00:00-04:00  207.910004  209.479996  204.259995  208.779999   
2025-05-20 00:00:00-04:00  207.669998  208.470001  205.029999  206.860001   
2025-05-21 00:00:00-04:00  205.169998  207.039993  200.710007  202.089996   
2025-05-22 00:00:00-04:00  200.710007  202.750000  199.699997  201.360001   
2025-05-23 00:00:00-04:00  193.669998  197.699997  193.460007  195.270004   

In [ ]:
#constraints

budget = 20000
risk_tolerance = 0.5

In [ ]:
#calculating the returns

In [2]:
import pandas as pd

tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'NFLX', 'AMD', 'IBM']
data = yf.download(tickers, start="2023-01-01", end="2025-01-01")

close_prices = data['Close']

close_prices = close_prices.pct_change().dropna()

covariance_matrix = close_prices.cov()
print(covariance_matrix)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  10 of 10 completed

Ticker      AAPL       AMD      AMZN     GOOGL       IBM      META      MSFT  \
Ticker                                                                         
AAPL    0.000181  0.000136  0.000101  0.000109  0.000025  0.000123  0.000097   
AMD     0.000136  0.000895  0.000245  0.000213  0.000081  0.000295  0.000199   
AMZN    0.000101  0.000245  0.000372  0.000204  0.000042  0.000269  0.000171   
GOOGL   0.000109  0.000213  0.000204  0.000339  0.000021  0.000225  0.000141   
IBM     0.000025  0.000081  0.000042  0.000021  0.000158  0.000053  0.000033   
META    0.000123  0.000295  0.000269  0.000225  0.000053  0.000579  0.000189   
MSFT    0.000097  0.000199  0.000171  0.000141  0.000033  0.000189  0.000204   
NFLX    0.000103  0.000231  0.000157  0.000123  0.000040  0.000166  0.000122   
NVDA    0.000147  0.000590  0.000249  0.000211  0.000059  0.000303  0.000225   
TSLA    0.000186  0.000329  0.000241  0.000196  0.000007  0.000227  0.000159   

Ticker      NFLX      NVDA      TSLA  


In [3]:
from qiskit import QuantumCircuit